# ATT&CK GPT

## Create ATT&CK Groups Knowledge Base

### Import Modules and Define Variables

In [4]:
from attackcti import attack_client
import os
import logging

logging.getLogger('taxii2client').setLevel(logging.CRITICAL)

In [24]:
# Define a few variables
current_directory = os.path.dirname("__file__")
knowledge_directory = os.path.join(current_directory, "knowledge")
db_directory = os.path.join(current_directory, "db")
templates_directory = os.path.join(current_directory, "templates")
group_template = os.path.join(templates_directory, "group.md")

### Initialize ATT&CK Client

In [ ]:
lift = attack_client()

### Get ATT&CK Groups Knowledge
Gettings technique STIX objects used by all groups accross all ATT&CK matrices..

In [ ]:
techniques_used_by_groups = lift.get_techniques_used_by_all_groups()
techniques_used_by_groups[0]

### Create ATT&CK Groups Documents

In [ ]:
import copy
from jinja2 import Template

# Create Group docs
all_groups = dict()
for technique in techniques_used_by_groups:
    if technique['id'] not in all_groups:
        group = dict()
        group['group_name'] = technique['name']
        group['group_id'] = technique['external_references'][0]['external_id']
        group['created'] = technique['created']
        group['modified'] = technique['modified']
        group['description'] = technique['description']
        group['aliases'] = technique['aliases']
        if 'x_mitre_contributors' in technique:
            group['contributors'] = technique['x_mitre_contributors']
        group['techniques'] = []
        all_groups[technique['id']] = group
    technique_used = dict()
    technique_used['matrix'] = technique['matrix']
    technique_used['domain'] = technique['x_mitre_domains']
    technique_used['platform'] = technique['platform']
    technique_used['tactics'] = technique['tactic']
    technique_used['technique_id'] = technique['technique_id']
    technique_used['technique_name'] = technique['technique']
    technique_used['use'] = technique['relationship_description']
    if 'data_sources' in technique:
        technique_used['data_sources'] = technique['data_sources']
    all_groups[technique['id']]['techniques'].append(technique_used)

print("[+] Creating markadown files for each group..")
markdown_template = Template(open(group_template).read())
for key in list(all_groups.keys()):
    group = all_groups[key]
    print("  [>>] Creating markdown file for {}..".format(group['group_name']))
    group_for_render = copy.deepcopy(group)
    markdown = markdown_template.render(metadata=group_for_render, group_name=group['group_name'], group_id=group['group_id'])
    file_name = (group['group_name']).replace(' ','_')
    open(f'{knowledge_directory}/{file_name}.md', encoding='utf-8', mode='w').write(markdown)

## Generate Knowledge Base Embeddings

### Load Documents

In [ ]:
import glob
from langchain.document_loaders import UnstructuredMarkdownLoader
import tiktoken

In [ ]:
# variables
group_files = glob.glob(os.path.join(knowledge_directory, "*.md"))

# Loading Markdown files
md_docs = []
print("[+] Loading Group markdown files..")
for group in group_files:
    print(f' [*] Loading {os.path.basename(group)}')
    loader = UnstructuredMarkdownLoader(group)
    md_docs.extend(loader.load())

print(f'[+] Number of .md documents processed: {len(md_docs)}')

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken.encoding_for_model('gpt-3.5-turbo')
token_counts = [tiktoken_len(doc.page_content) for doc in md_docs]

print(f"""[+] Token Counts:
Min: {min(token_counts)}
Avg: {int(sum(token_counts) / len(token_counts))}
Max: {max(token_counts)}""")

### Split Documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from tqdm.auto import tqdm
import hashlib

In [ ]:
# Chunking Text
print('[+] Initializing RecursiveCharacterTextSplitter..')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,  # number of tokens overlap between chunks
    length_function=tiktoken_len,
    separators=['\n\n', '\n', ' ', '']
)

print('[+] Splitting documents in chunks..')
chunks = text_splitter.split_documents(md_docs)
print(type(chunks))
print(len(chunks))

In [ ]:
print('[+] Splitting text in chunks..')
json_documents = []
chunks_documents = []
m = hashlib.md5()
for doc in tqdm(md_docs):
    doc_name = os.path.basename(doc.metadata['source'])
    m.update(doc_name.encode('utf-8'))
    uid = m.hexdigest()[:12]
    chunks = text_splitter.split_text(doc.page_content)
    for i, chunk in enumerate(chunks):
        # Add JSON object to array
        json_documents.append({
        'id': f'{uid}-{i}',
        'text': chunk,
        'source': doc_name
        })
        # Create docs
        document = Document(page_content=chunk, metadata={"source": doc_name})
        chunks_documents.append(document)

print(f'[+] Final Documents count: {len(json_documents)}')

### Export Knowledge Base as JSONL File (Optional)

In [ ]:
import json

In [ ]:
print(f'[+] Exporting groups as .jsonl file..')
with open(f'{os.path.join(db_directory, "attack-groups.jsonl")}', 'w') as f:
    for doc in json_documents:
        f.write(json.dumps(doc) + '\n')

### Generate Embeddings

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import openai
from dotenv import load_dotenv

In [ ]:
# Get your key: https://platform.openai.com/account/api-keys
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
print("[+] Starting embedding..")
embeddings = OpenAIEmbeddings()

# Send text chunks to OpenAI Embeddings API
print("[+] Sending chunks to OpenAI Embeddings API..")
db = FAISS.from_documents(chunks_documents, embeddings)

### Create Database Pickle File (Optional)

In [25]:
import pickle
import os

current_directory = os.path.dirname("__file__")
db_directory = os.path.join(current_directory, "db")

In [11]:
db_file = os.path.join(db_directory,"db.pkl")

if os.path.exists(db_file):
    with open(db_file, "rb") as f:
        db = pickle.load(f)
else:
    # Save vectorstore
    print("[+] Create Pickle file..")
    with open(db_file, "wb") as f:
        pickle.dump(db, f)

print(type(db))

## Query ATT&CK Groups Knowledge Base

### Initialize OpenAI (Optional)

In [17]:
from dotenv import load_dotenv
import openai

# Get your key: https://platform.openai.com/account/api-keys
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

### Define Vector Store Retriever
The retriever interface is a generic interface that makes it easy to combine documents with language models. This interface exposes a get_relevant_documents method which takes in a query (a string) and returns a list of documents.

In [ ]:
retriever = db.as_retriever(search_kwargs={"k":10})

### Get Relevant Documents

In [28]:
query = "What are some phishing techniques used by threat actors?"

In [29]:
print("[+] Getting relevant documents for query..")
relevant_docs = retriever.get_relevant_documents(query)
relevant_docs

[+] Getting relevant documents for query..


[Document(page_content='APT39 leveraged spearphishing emails with malicious attachments to initially compromise victims.(Citation: FireEye APT39 Jan 2019)(Citation: Symantec Chafer February 2018)(Citation: FBI FLASH APT39 September 2020)|', metadata={'source': 'APT39.md'}),
 Document(page_content='Lazarus Group has been observed targeting organizations using spearphishing documents with embedded malicious payloads. (Citation: Novetta Threat Research Group February 2016) Highly targeted spear phishing campaigns have been conducted against a U.S. electric grid company. (Citation: Eduard Kovacs March 2018)|', metadata={'source': 'Lazarus_Group.md'}),
 Document(page_content='TA505 has used spearphishing emails with malicious attachments to initially compromise victims.(Citation: Proofpoint TA505 Sep 2017)(Citation: Proofpoint TA505 June 2018)(Citation: Proofpoint TA505 Jan 2019)(Citation: Cybereason TA505 April 2019)(Citation: ProofPoint SettingContent-ms July 2018)(Citation: Proofpoint TA

### Question Answering

In [26]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [30]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
chain.run(input_documents=relevant_docs, question=query)

' Threat actors have used spearphishing emails with malicious attachments, links to HTML application files embedded with malicious code, links to malicious zip files, and messages with links designed to steal credentials or track victims.'